# How to add thread-level memory to a ReAct Agent

<div class="admonition tip">
    <p class="admonition-title">Prerequisites</p>
    <p>
        This guide assumes familiarity with the following:
        <ul>
            <li>            
                <a href="https://langchain-ai.github.io/langgraph/concepts/persistence/">
                    LangGraph Persistence
                </a>
            </li>
            <li>            
                <a href="https://langchain-ai.github.io/langgraph/concepts/persistence/#checkpointer-interface">
                    Checkpointer interface
                </a>
            </li>
            <li>
                <a href="https://langchain-ai.github.io/langgraph/concepts/agentic_concepts/">
                    Agent Architectures
                </a>                   
            </li>
            <li>
                <a href="https://python.langchain.com/docs/concepts/chat_models/">
                    Chat Models
                </a>
            </li>
            <li>
                <a href="https://python.langchain.com/docs/concepts/tools/">
                    Tools
                </a>
            </li>
        </ul>
    </p>
</div> 

This guide will show how to add memory to the prebuilt ReAct agent. Please see [this tutorial](../create-react-agent) for how to get started with the prebuilt ReAct agent

We can add memory to the agent, by passing a [checkpointer](https://langchain-ai.github.io/langgraph/reference/checkpoints/) to the [create_react_agent](https://langchain-ai.github.io/langgraph/reference/prebuilt/#langgraph.prebuilt.chat_agent_executor.create_react_agent) function.

## Setup

First, let's install the required packages and set our API keys

In [1]:
%%capture --no-stderr
%pip install -U langgraph langchain-openai

In [2]:
import getpass
import os


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("OPENAI_API_KEY")

OPENAI_API_KEY:  ········


<div class="admonition tip">
    <p class="admonition-title">Set up <a href="https://smith.langchain.com">LangSmith</a> for LangGraph development</p>
    <p style="padding-top: 5px;">
        Sign up for LangSmith to quickly spot issues and improve the performance of your LangGraph projects. LangSmith lets you use trace data to debug, test, and monitor your LLM apps built with LangGraph — read more about how to get started <a href="https://docs.smith.langchain.com">here</a>. 
    </p>
</div>

## Code

In [3]:
# First we initialize the model we want to use.
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o", temperature=0)

# For this tutorial we will use custom tool that returns pre-defined values for weather in two cities (NYC & SF)

from langchain_core.tools import tool


@tool
def get_weather(location: str) -> str:
    """Use this to get weather information."""
    if any([city in location.lower() for city in ["nyc", "new york city"]]):
        return "It might be cloudy in nyc"
    elif any([city in location.lower() for city in ["sf", "san francisco"]]):
        return "It's always sunny in sf"
    else:
        return f"I am not sure what the weather is in {location}"


tools = [get_weather]

# We can add "chat memory" to the graph with LangGraph's Redis checkpointer
# to retain the chat context between interactions
from langgraph.checkpoint.redis import RedisSaver

# Set up Redis connection
REDIS_URI = "redis://redis:6379"
memory = None
with RedisSaver.from_conn_string(REDIS_URI) as cp:
    cp.setup()
    memory = cp

# Define the graph

from langgraph.prebuilt import create_react_agent

graph = create_react_agent(model, tools=tools, checkpointer=memory)

00:25:12 langgraph.checkpoint.redis INFO   Redis client is a standalone client
00:25:12 redisvl.index.index INFO   Index already exists, not overwriting.
00:25:12 redisvl.index.index INFO   Index already exists, not overwriting.
00:25:12 redisvl.index.index INFO   Index already exists, not overwriting.


/tmp/ipykernel_1924/1444546798.py:39: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  graph = create_react_agent(model, tools=tools, checkpointer=memory)


## Usage

Let's interact with it multiple times to show that it can remember

In [4]:
def print_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()

In [5]:
config = {"configurable": {"thread_id": "1"}}
inputs = {"messages": [("user", "What's the weather in NYC?")]}

print_stream(graph.stream(inputs, config=config, stream_mode="values"))

================================ Human Message =================================

What's the weather in NYC?
00:25:13 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
================================== Ai Message ==================================
Tool Calls:
  get_weather (call_3ipbdgl0CGTjmjZmWDzwdDUR)
 Call ID: call_3ipbdgl0CGTjmjZmWDzwdDUR
  Args:
    location: New York City
================================= Tool Message =================================
Name: get_weather

It might be cloudy in nyc
00:25:14 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
================================== Ai Message ==================================

The weather in New York City might be cloudy.


Notice that when we pass the same thread ID, the chat history is preserved.

In [6]:
inputs = {"messages": [("user", "What's it known for?")]}
print_stream(graph.stream(inputs, config=config, stream_mode="values"))

================================ Human Message =================================

What's it known for?
00:25:21 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
================================== Ai Message ==================================

New York City is known for many things, including:

1. **Landmarks and Attractions**: The Statue of Liberty, Times Square, Central Park, Empire State Building, and Broadway theaters.
   
2. **Cultural Diversity**: A melting pot of cultures, languages, and cuisines from around the world.

3. **Financial Hub**: Home to Wall Street and the New York Stock Exchange, making it a global financial center.

4. **Arts and Entertainment**: Renowned museums like the Metropolitan Museum of Art and the Museum of Modern Art, as well as a vibrant music and theater scene.

5. **Fashion**: A major fashion capital, hosting New York Fashion Week and housing numerous designer boutiques.

6. **Cuisine**: Famous for its divers